In [1]:
from transformers import GPT2TokenizerFast, GPT2Tokenizer
from copy import deepcopy
import numpy as np
from random import random
from time import time

/home/maciej/anaconda3/envs/tokenizex/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

from data_loader import get_imput
from embedding_mask import create_mask_embedding, sum_masks, process_token, encode_em


In [3]:
# tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
vocab = tokenizer.get_vocab()
vocabi = {v:k for k, v in vocab.items()}
SRC = "asd_pol.txt"
with open(SRC, "r") as f:
    all_txt = f.read()
def get_imput(txt, length=512):
    bi = int(random()*(len(txt)-1000))
    return txt[bi:bi+length]

/home/maciej/anaconda3/envs/tokenizex/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:

def tokenize_atoms(txt, tokenizer):
    atxt = "".join(tokenizer.tokenize(txt))
    atxt = list(atxt)
    eatxt = tokenizer.encode(atxt)
    return eatxt

c = 0

def lazy_past_tokens_gather(atokens, tokenizer): #dev no dobra, może tak być dla tokenów kłopotliwych, ale dla reszty już nie powinno
    global c #dev
    c+=1 #dev
    past_emb_pos = []
    for i in range(1, len(atokens)+1):
        past_emb_pos.append(atokens[:i])
    return past_emb_pos

def past_tokens_gather(atokens, tokenizer):
    past_emb_pos = []
    for i in range(1, len(atokens)+1):
        pdec = tokenizer.decode(atokens[:i])
        if '�' in pdec:
            return lazy_past_tokens_gather(atokens, tokenizer)
        penc = tokenizer.encode(pdec)
        past_emb_pos.append(penc)
    return past_emb_pos

def create_emb_pos_mask(emb_pos):
    res = []
    # mask = np.eye(len(emb_pos)).astype(bool)
    mask = np.eye(len(emb_pos))
    for embt, m in zip(emb_pos, mask):
        res.append(embt[-1])
        for ie in embt[:-1]:
            is_added = True if res[:-1] == [] else False
            for j, je in enumerate(res[:-1]):
                if ie == je:
                    # m[j] = True
                    m[j] = 1
                    is_added = True
                    break
            if not is_added:
                raise Exception("target len != input len")
    return np.array(res)[:,0], np.array(res)[:,1], mask

def tokenizex_encode_full(txt, tokenizer):
    tenc = tokenize_atoms(txt, tokenizer)
    res = past_tokens_gather(tenc, tokenizer)
    g = [list(zip(e, np.arange(len(e)))) for e in res]
    e, p, m = create_emb_pos_mask(g)
    return e, p, m


In [5]:
def tokenizex_encode(txt, tokenizer, atomizer):
    tokens = tokenizer.encode(txt)

    res_emb = []
    res_pos = []
    partial_masks = []
    pos_len = 0
    
    for t in tokens:
        emb, pos, mask = atomizer[t]
        res_emb.extend(emb)
        res_pos.extend(pos+pos_len)
        pos_len += pos[-1] + 1
        partial_masks.append(mask)   
    
    full_mask = sum_masks(len(res_emb), partial_masks)
    return np.array(res_emb), np.array(res_pos), full_mask


In [80]:
def validate(txt, emb, pos, mask, tokenizer):
    dec = tokenizer.decode(emb[mask[-1].astype(bool)])
    assert dec == txt

    lemb = tokenize_atoms(txt, tokenizer)
    assert len(lemb) == len(emb)

    comp = pos[mask[-1].astype(bool)]
    for a, b in zip(comp.astype(int), list(range(len(comp)))):
        assert a == b

In [81]:
def tokenizex_w_encode(txt, tokenizer):
    res_emb = []
    res_pos = []
    partial_masks = []
    pos_len = 0
    
    txtw = [" " + e for e in txt.split(" ")]
    if len(txtw[0]) == 1:
        del txtw[0]
    else:
        txtw[0] = txtw[0][1:]
    
    for w in txtw:
        emb, pos, mask = tokenizex_encode_full(w, tokenizer)
        res_emb.extend(emb)
        res_pos.extend(pos+pos_len)
        pos_len += pos[-1] + 1
        partial_masks.append(mask)   
    full_mask = sum_masks(len(res_emb), partial_masks)
    return np.array(res_emb), np.array(res_pos), full_mask

def tokenizex_t_encode(txt, tokenizer):
    res_emb = []
    res_pos = []
    partial_masks = []
    pos_len = 0
    
    tok = encoder.encode(txt)
    
    for w in txtw:
        emb, pos, mask = tokenizex_encode_full(w, tokenizer)
        res_emb.extend(emb)
        res_pos.extend(pos+pos_len)
        pos_len += pos[-1] + 1
        partial_masks.append(mask)   
    full_mask = sum_masks(len(res_emb), partial_masks)
    return np.array(res_emb), np.array(res_pos), full_mask


In [82]:
import re


def remove_whitespace_before_dots(text):
    # Używamy wyrażenia regularnego, aby zastąpić białe znaki przed kropkami
    modified_text = re.sub(r'\s+\.', '.', text)
    return modified_text

In [83]:
txt = " return np.array(res_emb), np.array(res_pos), full_mask"
txt = """  𡨸 𡨸żóąs"""
txt = """cial Maurice Rossel reported that conditions there were "almost normal".
2014 – Under the terms of UN Security C"""
txt = tokenizer.prepare_for_tokenization(txt)[0]
txt = remove_whitespace_before_dots(txt)

In [90]:
txt = errorous
txt = """\n\nY"""

In [91]:
tokenizer.encode(txt)

[198, 198, 56]

In [93]:
tokenizer.decode([628, 56])


'\n\nY'

In [86]:
tenc = tokenize_atoms(txt, tokenizer)
print(tenc)
res = past_tokens_gather(tenc, tokenizer)
print(res)
g = [list(zip(e, np.arange(len(e)))) for e in res]
e, p, m = create_emb_pos_mask(g)
print(e)
validate(txt, e, p, m, tokenizer)

[198, 198, 56]
[[198], [628], [198, 198, 56]]


Exception: target len != input len

In [85]:
e, p, m = tokenizex_encode_full(txt, tokenizer)
validate(txt, e, p, m, tokenizer)

Exception: target len != input len

In [67]:
ee, pp, mm = tokenizex_w_encode(txt, tokenizer)
# print(ee)
# print(mm)
validate(txt, ee, pp, mm, tokenizer)

Exception: target len != input len

In [13]:
def tokenizer_def(txt, tokenizer):
    return tokenizer.encode(txt), None, None

def tokenizer_my1(txt, tokenizer):
    tenc = tokenize_atoms(txt, tokenizer)
    res = past_tokens_gather(tenc, tokenizer)
    g = [list(zip(e, np.arange(len(e)))) for e in res]
    e, p, m = create_emb_pos_mask(g)
    return e, p, m

def tokenizer_my2(txt, tokenizer):
    e, p, m= tokenizex_w_encode(txt, tokenizer)
    return e, p, m

In [21]:
errorous = ""
def time_valid_tokenization(txt_data, tokenizer, def_tokenizer, C, lbl_val = False):
    st = 0
    global errorous
    for i in range(C):
        txt = get_imput(txt_data, 112)
        # txti = def_tokenizer.decode(def_tokenizer.encode(get_imput(txt_data, 112)))
        txt = def_tokenizer.prepare_for_tokenization(txt)[0]
        txt = remove_whitespace_before_dots(txt)
        t1 = time()
        errorous = txt
        emb, pos, mask = tokenizer(txt, def_tokenizer)
        st += time()-t1

        if lbl_val: 
            print("--------------------")
            print(txt)
            validate(txt, emb, pos, mask, def_tokenizer)

    return st


In [22]:
time_valid_tokenization(all_txt, tokenizer_def, tokenizer, 100, False)

0.017826080322265625

In [23]:
time_valid_tokenization(all_txt, tokenizer_my2, tokenizer, 100, True)

--------------------
From today's featured article
Cyclone Taylor
Cyclone Taylor (June 23, 1884 – June 9, 1979) was a Canadian profes
--------------------
ie Mays dies at the age of 93.
In basketball, the Boston Celtics defeat the Dallas Mavericks to win the NBA Fina
--------------------
logo Wikiquote
Collection of quotations
Wikisource logo Wikisource
Free-content library
Wikispecies logo Wikispe
--------------------
Recent deaths: James K. IrvingGerhard KlingenbergDaniel Patrick ReillyAnouk AiméeDonald SutherlandMike Brumley
N


Exception: target len != input len

In [29]:
time_valid_tokenization(all_txt, tokenizer_my1, tokenizer, 100, True)

--------------------
yclone Taylor (June 23, 1884 – June 9, 1979) was a Canadian professional ice hockey player and civil servant. Bo
--------------------
the Renfrew Creamery Kings, becoming the highest-paid athlete in the world on a per-game basis. He then played f
--------------------
on 2118, the last of Syria's declared chemical weapons were shipped out for destruction.
Aymer de Valence, 2nd E
--------------------
rpoint)?... that because Larrabee County was not established, Iowa remains a state with 99 counties?... that Y
--------------------
 nowym numerem @anatomja x @cukiercukier ❤️ · NOWY ROK · Hej! Zapraszamy Cię! Jeżeli uważasz...

ZIOMECZKI ZIOM
--------------------
base
Wikinews logo Wikinews
Free-content news
Wikiquote logo Wikiquote
Collection of quotations
Wikisource logo 
--------------------
e statementMobile viewWikimedia FoundationPowered by MediaWiki
Przejdź do głównej zawartościUłatwienia dostępu –
--------------------
artościUłatwienia dostępu – pomoc
Opinia o u

Exception: target len != input len

In [ ]:
atomizer[120]

(array([171, 123, 121]),
 array([0, 1, 2]),
 array([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]]))

In [ ]:
txt = """ ósiem"""
emb, pos, mask = encode_em(txt, tokenizer)
validate(txt, emb, pos, mask, tokenizer)

AssertionError: 

In [ ]:
txt = " ósiem"

tokens = tokenizer.tokenize(txt)
res_emb = []
res_pos = []
partial_masks = []
pos_len = 0

for t in tokens:
    emb, pos, mask = process_token(t, tokenizer)

    res_emb.extend(emb)
    res_pos.extend(pos+pos_len)
    pos_len += pos[-1] + 1
    partial_masks.append(mask)



In [ ]:
t1  = "elo źle żal ósemka no i śląska"
t3 = "example text to tokenize and, żargon ołówka źle wpływa detokenize, ślóza."
t1  = "elo źle żaló"
t3 = "example."
e2  = np.array([69])
p2  = np.array([0])
m2  = np.array([1])
t2 = tokenizer.decode(e2)

e1, p1, m1 = encode_em(t1, tokenizer)
e3, p3, m3 = encode_em(t3, tokenizer)
txt = t1+t2+t3
txt

emb = np.concatenate([e1,e2,e3])
pos = sum_pos([p1,p2,p3])
mlen = m1.shape[0]+m2.shape[0]+m3.shape[0]
mask = sum_masks(mlen, [m1, m2, m3])


In [ ]:
# faulty examples
' ósiem'
"ó"
""" ›"""
''

In [ ]:

def text_emb_pos(txt, tokenizer):
    emb_pos = []

    letter_split = False
    for l in txt:
        t = tokenizer.encode(l)
        if len(t) > 1:
            letter_split = True
    
    if letter_split:
        for l in txt:
            et = tokenizer.encode(l)
            emb_pos.append(list(zip(et, list(range(len(et))))))
    else:
        for i in range(1, len(txt)+1):
            et = tokenizer.encode(txt[:i])
            emb_pos.append(list(zip(et, list(range(len(et))))))

    return emb_pos

In [ ]:
for t in atomizer.keys():
    txt = tokenizer.decode(t)
    emb = tokenize_letters(txt, tokenizer)
    a_emb = atomizer[t][0]
    assert len(emb) == len(a_emb)
    if len(emb) != len(a_emb):
        print(len(emb), len(a_emb))

In [ ]:
def tokenize_letters(txt, tokenizer):
    emb = []
    for l in txt:
        emb.extend(tokenizer.encode(l))
    return emb
